# Comparación de Modelos de Sentimiento

Este notebook compara el rendimiento de un modelo de Regresión Logística con TF-IDF (entrenado localmente) contra un modelo de Transformers pre-entrenado (RoBERTa) de la librería Hugging Face, especializado en análisis de sentimiento en tweets.

In [1]:
import joblib
import re
import spacy
import os
import pandas as pd
from transformers import pipeline

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.width', 200)

### Se define la función de limpieza de texto y una nueva función para encapsular la predicción con el modelo local.

In [2]:
# Cargar el modelo de spaCy
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Descargando modelo de spaCy...")
    os.system('python -m spacy download en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')


def limpieza(texto):
    # Realiza una limpieza completa del texto para el modelo local
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúüñ\s]', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    if not texto:
        return ""
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop])

def predecir_con_modelo_local(textos, vectorizer, modelo):
    # Encapsula todo el proceso de predicción del modelo local.
    textos_limpios = [limpieza(t) for t in textos]
    textos_transformados = vectorizer.transform(textos_limpios)
    predicciones = modelo.predict(textos_transformados)
    
    mapa_local = {-1: "Negativo", 1: "Positivo"}
    return [mapa_local.get(p, "Desconocido") for p in predicciones]

### Carga, Predicción y Comparación

In [3]:
try:
    #  Cargar Modelo Local
    vectorizer_cargado = joblib.load('model_tf_idf/schimizzi_vectorizer_1.6kk.joblib')
    modelo_cargado = joblib.load('model_tf_idf/schimizzi_modelo_1.6kk.joblib')
    print("Modelo local cargado!")

    #  Cargar roBERTa
    sentiment_pipeline_hf = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    print("roBERTa de Hugging Face cargado!")

    #  Cargar Datos para Comparación 
    ruta_archivo_nuevo = 'data/testdata.manual.2009.06.14.csv'
    df_comparacion = pd.read_csv(ruta_archivo_nuevo, header=None, encoding='latin-1')
    df_comparacion.columns = ['Sentiment_Real', 'id', 'date', 'query', 'user', 'Text']
    
    # Tomamos una muestra para no tardar demasiado
    textos_a_comparar = df_comparacion['Text'].sample(30).tolist() # una muestra aleatoria de 30 tweets
    
    #  Realizar Predicciones 
    print("\nRealizando predicciones con ambos modelos...")
    predicciones_local = predecir_con_modelo_local(textos_a_comparar, vectorizer_cargado, modelo_cargado)
    resultados_hf = sentiment_pipeline_hf(textos_a_comparar)
    
    #  Formatear Resultados y Mostrar Tabla 
    mapa_hf = {'LABEL_0': 'Negativo', 'LABEL_1': 'Neutral', 'LABEL_2': 'Positivo'}
    predicciones_hf_texto = [mapa_hf.get(r['label'], "Desconocido") for r in resultados_hf]
    confianza_hf = [r['score'] for r in resultados_hf]

    df_resultados = pd.DataFrame({
        'Texto Original': textos_a_comparar,
        'Modelo Local': predicciones_local,
        'roBERTa': predicciones_hf_texto,
        'Confianza HF': confianza_hf
    })

    print("\n-------------------------------- TABLA COMPARATIVA DE PREDICCIONES -------------------------------- ")
    print(df_resultados.to_string())

except FileNotFoundError:
    print("\nError: No se encontraron los archivos del modelo o el archivo de datos.")
except Exception as e:
    print(f"\nOcurrió un error inesperado: {e}")

Modelo local cargado!


Device set to use cpu


roBERTa de Hugging Face cargado!

Realizando predicciones con ambos modelos...

-------------------------------- TABLA COMPARATIVA DE PREDICCIONES -------------------------------- 
                                                                                                                                      Texto Original Modelo Local   roBERTa  Confianza HF
0                                                                            Awesome viral marketing for "Funny People" http://www.nbc.com/yo-teach/     Positivo  Positivo      0.961671
1                                                            Share: Disruption...Fred Wilson's slides for his talk at Google HQ  http://bit.ly/Bo8PG     Positivo   Neutral      0.723739
2                                    I hate revision, it's so boring! I am totally unprepared for my exam tomorrow :( Things are not looking good...     Negativo  Negativo      0.984168
3              My dad was in NY for a day, we ate at MESA grill last night 

### Análisis de Resultados

Al observar la tabla comparativa, se pueden extraer las siguientes conclusiones:

- Matices y Contexto: El modelo de Hugging Face (roBERTa) es un Transformer, por lo que es mucho mejor para entender el contexto, el sarcasmo y los matices del lenguaje que un modelo TF-IDF. 
- Manejo de la Neutralidad: Este modelo solo puede clasificar como "Positivo" o "Negativo". El modelo de Hugging Face tiene una clase "Neutral", lo que le da más flexibilidad y precisión en textos que no tienen una carga sentimental clara.
- Confianza del Modelo: La columna 'Confianza HF' indica qué tan seguro está el modelo de su predicción. En textos ambiguos, esta confianza suele ser más baja.

#### ❤   Estoy muy conforme con lo conseguido con mi modelo de predicción de sentimiento!   ❤